<a href="https://colab.research.google.com/github/oddrationale/AdventOfCode2023Python/blob/main/Day09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<article class="day-desc"><h2>--- Day 9: Mirage Maintenance ---</h2><p>You ride the camel through the sandstorm and stop where the ghost's maps told you to stop. <span title="The sound of a sandstorm slowly settling.">The sandstorm subsequently subsides, somehow seeing you standing at an <em>oasis</em>!</span></p>
<p>The camel goes to get some water and you stretch your neck. As you look up, you discover what must be yet another giant floating island, this one made of metal! That must be where the <em>parts to fix the sand machines</em> come from.</p>
<p>There's even a <a href="https://en.wikipedia.org/wiki/Hang_gliding" target="_blank">hang glider</a> partially buried in the sand here; once the sun rises and heats up the sand, you might be able to use the glider and the hot air to get all the way up to the metal island!</p>
<p>While you wait for the sun to rise, you admire the oasis hidden here in the middle of Desert Island. It must have a delicate ecosystem; you might as well take some ecological readings while you wait. Maybe you can report any environmental instabilities you find to someone so the oasis can be around for the next sandstorm-worn traveler.</p>
<p>You pull out your handy <em>Oasis And Sand Instability Sensor</em> and analyze your surroundings. The OASIS produces a report of many values and how they are changing over time (your puzzle input). Each line in the report contains the <em>history</em> of a single value. For example:</p>
<pre><code>0 3 6 9 12 15
1 3 6 10 15 21
10 13 16 21 30 45
</code></pre>
<p>To best protect the oasis, your environmental report should include a <em>prediction of the next value</em> in each history. To do this, start by making a new sequence from the <em>difference at each step</em> of your history. If that sequence is <em>not</em> all zeroes, repeat this process, using the sequence you just generated as the input sequence. Once all of the values in your latest sequence are zeroes, you can extrapolate what the next value of the original history should be.</p>
<p>In the above dataset, the first history is <code>0 3 6 9 12 15</code>. Because the values increase by <code>3</code> each step, the first sequence of differences that you generate will be <code>3 3 3 3 3</code>. Note that this sequence has one fewer value than the input sequence because at each step it considers two numbers from the input. Since these values aren't <em>all zero</em>, repeat the process: the values differ by <code>0</code> at each step, so the next sequence is <code>0 0 0 0</code>. This means you have enough information to extrapolate the history! Visually, these sequences can be arranged like this:</p>
<pre><code>0   3   6   9  12  15
  3   3   3   3   3
    0   0   0   0
</code></pre>
<p>To extrapolate, start by adding a new zero to the end of your list of zeroes; because the zeroes represent differences between the two values above them, this also means there is now a placeholder in every sequence above it:<p>
<pre><code>0   3   6   9  12  15   <em>B</em>
  3   3   3   3   3   <em>A</em>
    0   0   0   0   <em>0</em>
</code></pre>
<p>You can then start filling in placeholders from the bottom up. <code>A</code> needs to be the result of increasing <code>3</code> (the value to its left) by <code>0</code> (the value below it); this means <code>A</code> must be <code><em>3</em></code>:</p>
<pre><code>0   3   6   9  12  15   B
  3   3   3   3   <em>3</em>   <em>3</em>
    0   0   0   0   <em>0</em>
</code></pre>
<p>Finally, you can fill in <code>B</code>, which needs to be the result of increasing <code>15</code> (the value to its left) by <code>3</code> (the value below it), or <code><em>18</em></code>:</p>
<pre><code>0   3   6   9  12  <em>15</em>  <em>18</em>
  3   3   3   3   3   <em>3</em>
    0   0   0   0   0
</code></pre>
<p>So, the next value of the first history is <code><em>18</em></code>.</p>
<p>Finding all-zero differences for the second history requires an additional sequence:</p>
<pre><code>1   3   6  10  15  21
  2   3   4   5   6
    1   1   1   1
      0   0   0
</code></pre>
<p>Then, following the same process as before, work out the next value in each sequence from the bottom up:</p>
<pre><code>1   3   6  10  15  21  <em>28</em>
  2   3   4   5   6   <em>7</em>
    1   1   1   1   <em>1</em>
      0   0   0   <em>0</em>
</code></pre>
<p>So, the next value of the second history is <code><em>28</em></code>.</p>
<p>The third history requires even more sequences, but its next value can be found the same way:</p>
<pre><code>10  13  16  21  30  45  <em>68</em>
   3   3   5   9  15  <em>23</em>
     0   2   4   6   <em>8</em>
       2   2   2   <em>2</em>
         0   0   <em>0</em>
</code></pre>
<p>So, the next value of the third history is <code><em>68</em></code>.</p>
<p>If you find the next value for each history in this example and add them together, you get <code><em>114</em></code>.</p>
<p>Analyze your OASIS report and extrapolate the next value for each history. <em>What is the sum of these extrapolated values?</em></p>
</article>

In [1]:
from google.colab import drive
drive.mount(r"/content/drive")

input_file = r"/content/drive/MyDrive/Colab Notebooks/AoC2023/input/09.txt"
with open(input_file, "r") as file:
    input = file.read()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
def parse_input(input: str) -> list[list[int]]:
    """
    Parse a multiline string into a list of lists of integers.

    Each line in the input string is assumed to be a sequence of integers
    separated by spaces. The function converts these lines into lists of integers.

    Args:
    input (str): A multiline string where each line represents a sequence of integers.

    Returns:
    list[list[int]]: A list of lists, where each sublist is a sequence of integers from the input.
    """

    # Split the input string into lines
    lines = input.splitlines()

    # Convert each line into a list of integers
    sequences = [list(map(int, line.split())) for line in lines if line]

    return sequences

In [3]:
def extrapolate_next(history: list[int]) -> int:
    """
    Given a history of integers, recursively extrapolate the next integer in the sequence.

    The function calculates differences between consecutive numbers recursively and uses
    these differences to predict the next number. The recursion continues until
    a sequence of differences that are all zero is found.

    Args:
    history (list[int]): A list of integers representing the history of a sequence.

    Returns:
    int: The next integer in the sequence.
    """

    def calculate_next(history: list[int]) -> int:
        """
        Helper function to recursively calculate the next value in the sequence.
        """
        if len(history) == 1:
            return history[0]

        # Compute the difference sequence
        differences = [j - i for i, j in zip(history, history[1:])]

        # If all differences are zero, return the last element of the history
        if all(d == 0 for d in differences):
            return history[-1]

        # Recursively call the function to compute the next value in the difference sequence
        next_difference = calculate_next(differences)
        # Add the next difference to the last element of the current history to get the next value
        return history[-1] + next_difference

    if not history or len(history) < 2:
        raise ValueError("History must contain at least two values.")

    return calculate_next(history)

In [4]:
oasis_report = parse_input(input)

In [5]:
sum(extrapolate_next(history) for history in oasis_report)

1861775706

<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Of course, it would be nice to have <em>even more history</em> included in your report. Surely it's safe to just <em>extrapolate backwards</em> as well, right?</p>
<p>For each history, repeat the process of finding differences until the sequence of differences is entirely zero. Then, rather than adding a zero to the end and filling in the next values of each previous sequence, you should instead add a zero to the <em>beginning</em> of your sequence of zeroes, then fill in new <em>first</em> values for each previous sequence.</p>
<p>In particular, here is what the third example history looks like when extrapolating back in time:</p>
<pre><code><em>5</em>  10  13  16  21  30  45
  <em>5</em>   3   3   5   9  15
   <em>-2</em>   0   2   4   6
      <em>2</em>   2   2   2
        <em>0</em>   0   0
</code></pre>
<p>Adding the new values on the left side of each sequence from bottom to top eventually reveals the new left-most history value: <code><em>5</em></code>.</p>
<p>Doing this for the remaining example data above results in previous values of <code><em>-3</em></code> for the first history and <code><em>0</em></code> for the second history. Adding all three new values together produces <code><em>2</em></code>.</p>
<p>Analyze your OASIS report again, this time extrapolating the <em>previous</em> value for each history. <em>What is the sum of these extrapolated values?</em></p>
</article>

In [6]:
def extrapolate_previous(history: list[int]) -> int:
    """
    Given a history of integers, recursively extrapolate the previous integer in the sequence.

    The function calculates differences between consecutive numbers recursively and uses
    these differences to predict the previous number. The recursion continues until
    a sequence of differences that are all zero is found.

    Args:
    history (list[int]): A list of integers representing the history of a sequence.

    Returns:
    int: The previous integer in the sequence.
    """

    def calculate_previous(history: list[int]) -> int:
        """
        Helper function to recursively calculate the previous value in the sequence.
        """
        if len(history) == 1:
            return history[0]

        # Compute the difference sequence
        differences = [j - i for i, j in zip(history, history[1:])]

        # If all differences are zero, return the first element of the history
        if all(d == 0 for d in differences):
            return history[0]

        # Recursively call the function to compute the previous value in the difference sequence
        previous_difference = calculate_previous(differences)
        # Subtract the previous difference from the first element of the current history to get the previous value
        return history[0] - previous_difference

    if not history or len(history) < 2:
        raise ValueError("History must contain at least two values.")

    return calculate_previous(history)

In [7]:
sum(extrapolate_previous(history) for history in oasis_report)

1082